## PREDICT THE BURNED AREA OF FOREST FIRES WITH NEURAL NETWORKS

In [1]:
# pip install tensorflow

In [2]:
# pip install keras

In [3]:
!pip install --upgrade scikit-learn

In [4]:
! pip install imblearn

In [5]:
import imblearn

ImportError: cannot import name '_MissingValues' from 'sklearn.utils._param_validation' (C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\_param_validation.py)

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense,Dropout
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix
from keras.models import Sequential
from tensorflow.keras.optimizers import Adam,RMSprop
from sklearn.model_selection import GridSearchCV,KFold

In [ ]:
fire = pd.read_csv("P:/DS material/DS Assignments/Solved and unsolved assignments/All DS Assignments/All Datasets for assignments/forestfires.csv")
fire

### EDA

In [ ]:
fire.shape

In [ ]:
fire.info()

In [ ]:
fire.describe()

In [ ]:
fire.isnull()

In [ ]:
fire.isnull().sum()

In [ ]:
fire.columns

In [ ]:
sns.countplot(fire['size_category'])

In [ ]:
plt.figure(figsize=(12,10))
sns.countplot(x = fire['month'], hue = fire['size_category'])
plt.show()

In [ ]:
plt.figure(figsize=(12,10))
sns.countplot(x = fire['day'], hue = fire['size_category'])
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['FFMC'],color='green')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['DMC'],color='red')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['DC'],color='purple')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['ISI'],color='grey')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['temp'],color='pink')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['RH'],color='blue')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['wind'],color='yellow')
plt.show()

In [ ]:
plt.figure(figsize = (12,10))
sns.distplot(fire['rain'],color='orange')
plt.show()

In [ ]:
drop_col = fire.drop(labels = ['month', 'day'], axis = 1)
drop_col

In [ ]:
drop_col.shape

In [ ]:
from sklearn import preprocessing
labelEncoder = preprocessing.LabelEncoder()

In [ ]:
le = labelEncoder
drop_col['size_category'] = le.fit_transform(drop_col['size_category'])
drop_col

In [ ]:
sns.countplot(drop_col['size_category'])

In [ ]:
drop_col['size_category'].replace({'small':0,'large':0},inplace = True)
drop_col

In [ ]:
fire_corr = drop_col.corr()
fire_corr

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(fire_corr, annot = True)
plt.show()

In [ ]:
pair_data = pd.concat([drop_col.iloc[:,0:9], drop_col.iloc[:,-1]],axis = 1)
sns.pairplot(pair_data)
plt.show()

### Model Building

In [ ]:
x = drop_col.drop(labels='size_category',axis = 1)
y = drop_col[['size_category']]

In [ ]:
x

In [ ]:
y

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(x, y,
                                                   test_size = .3, random_state = 45)

In [ ]:
print("shape of x_train:", x_train.shape)
print("shape of y_train:", y_train.shape)

Data Is Imbalance so i have to balance it , so i m using here smote operation of balancing technique

In [ ]:
sm = SMOTE(random_state=12)
x_train_sm,y_train_sm = sm.fit_resample(x_train,np.array(y_train).ravel())
x_train_sm,y_train_sm

In [ ]:
x_train_sm.shape, y_train_sm.shape

In [ ]:
x_train = x_train_sm.copy()
y_train = y_train_sm.copy()

### Converting Data into Standard Scale

In [ ]:
scale = MinMaxScaler()
X_train = scale.fit_transform(x_train)
X_train

In [ ]:
X_test = scale.fit_transform(x_test)
X_test

### Model Training

Tuning of Hyperparameter : Batch size and Epoch

In [ ]:
def creat_model():
    model = Sequential()
    model.add(Dense(8, input_dim = 28,kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4,kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1,kernel_initializer='uniform', activation='sigmoid'))
    adam = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy',optimizer = adam,metrics='accuracy')
    return model

In [ ]:
model = KerasClassifier(build_fn=creat_model,verbose = 0)
batch_size = [10,30,50]
epochs = [10,20,50]
param_grid = dict(batch_size = batch_size,epochs = epochs)
gsv = GridSearchCV(estimator=model,param_grid=param_grid,cv = KFold(),verbose=5)
gsv_res = gsv.fit(X_train,y_train)

In [ ]:
print(gsv_res.best_params_,gsv_res.best_score_)

In [ ]:
#Turning Hyperparameter: Learning rate and Dropout rate
def creat_model(learning_rate,dropout_rate):
    model = Sequential()
    model.add(Dense(8, input_dim = 28,kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(4,kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(1,kernel_initializer='uniform', activation='sigmoid'))
    adam = Adam(learning_rate=learning_rate)
    model.compile(loss='binary_crossentropy',optimizer = adam,metrics='accuracy')
    return model

In [ ]:
model = KerasClassifier(build_fn=creat_model,batch_size = 10,epochs = 50,verbose = 0)
learning_rate = [0.1,0.01,0.001]
dropout_rate = [0.0,0.1,0.2]
param_grid = dict(learning_rate = learning_rate,dropout_rate = dropout_rate)
gsv = GridSearchCV(estimator=model,param_grid=param_grid,cv= KFold(),verbose=5)
gsv_r = gsv.fit(X_train,y_train)

In [ ]:
print(gsv_r.best_params_,gsv_r.best_score_)

 Tuning of Hyperparameter :Activation Function & Kernel Initializer

In [ ]:
def creat_model(Activation_Function,init):
    model = Sequential()
    model.add(Dense(8, input_dim = 28,kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(4,kernel_initializer='uniform', activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(1,kernel_initializer='uniform', activation='sigmoid'))
    adam = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy',optimizer = adam,metrics='accuracy')
    return model

In [ ]:
model = KerasClassifier(build_fn=creat_model,batch_size = 10,epochs = 50,verbose = 0)
Activation_Function = ['relu','tanh','softmax','linear']
init = ['zero','uniform','normal']
param_grid = dict(Activation_Function = Activation_Function,init = init)

gsv = GridSearchCV(estimator=model,param_grid=param_grid,cv= KFold(),verbose=5)
gsv_result = gsv.fit(X_train,y_train)

In [ ]:
print(gsv_result.best_score_,gsv_result.best_params_)

Tuning of Hyperparameter :Number of Neurons in hidden layer

In [ ]:
def creat_model(neuron1,neuron2):
    model = Sequential()
    model.add(Dense(8,input_dim=28,kernel_initializer='normal', activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(4,kernel_initializer='normal',activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    adam = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=creat_model,batch_size = 10,epochs = 50,verbose = 0)
neuron1 = [24,16,8]
neuron2 = [12,8,4]
param_grid = dict(neuron1 = neuron1,neuron2=neuron2)
gsv = GridSearchCV(estimator=model,param_grid=param_grid,cv=KFold(),verbose=5)
gsv_n = gsv.fit(X_train,y_train)

In [ ]:
print(gsv_n.best_score_,gsv_n.best_params_)

Train a model with optimum values of hyperparameter

In [ ]:
# best Parameters
# batch_size = 10
# epochs = 50
# dropout_rate = 0.1
# learning rate = 0.001
# activation_function = tanh
# kernel_initiative = normal
# neuron1 = 8
# neuron2 = 8

In [ ]:
def creat_model():
    model = Sequential()
    model.add(Dense(8,input_dim=28,kernel_initializer='normal', activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(8,kernel_initializer='normal',activation='tanh'))
    model.add(Dropout(0.1))
    model.add(Dense(1,kernel_initializer='normal',activation='sigmoid'))
    adam = Adam(learning_rate=0.001)
    model.compile(loss='binary_crossentropy', optimizer=adam, metrics=['accuracy'])
    return model

In [ ]:
model = KerasClassifier(build_fn=creat_model,batch_size = 10,epochs = 50)
model.fit(X_train,y_train)

In [ ]:
y_predict = model.predict(X_train)

In [ ]:
accuracy_score(y_train,y_predict)

In [ ]:
confusion_matrix(y_train,y_predict)

In [ ]:
print(classification_report(y_train,y_predict))

In [ ]:
# testing data
y_test_pred = model.predict(X_test)

In [ ]:
accuracy_score(y_test,y_test_pred)

In [ ]:
confusion_matrix(y_test,y_test_pred)

In [ ]:
print(classification_report(y_test,y_test_pred))